In [105]:
import numpy as np
import pydot
from IPython.display import display, SVG, clear_output


from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.all import (Parser, Meshcat, DiagramBuilder,
                        MeshcatVisualizerCpp, JacobianWrtVariable,
                        MakeRenderEngineVtk, RenderEngineVtkParams)
from pydrake.geometry import (Box, Cylinder)
from pydrake.multibody.tree import (PrismaticJoint, UnitInertia, SpatialInertia, RevoluteJoint, FixedOffsetFrame, WeldJoint)
from manipulation.meshcat_cpp_utils import MeshcatJointSliders
from manipulation.scenarios import AddMultibodyTriad
meshcat = Meshcat()

[2022-03-31 01:00:21.926] [console] [info] Meshcat listening for connections at http://localhost:7001


In [106]:
# Build robot model in Drake
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)

# parameters
L0 = 0.8 #lenght of link 0
L1 = 1
L2 = 0.5
L3 = 0.08 # end effector

RGBA_Color0 = [0.5, 0.5, 0.5, 0.4]
RGBA_Color1 = [0, 0.5, 0.5, 0.4]
RGBA_Color2 = [0.9, 0, 0, 0.4]

my_model_instance = plant.AddModelInstance("my_robot")

#SpatialInertia(mass, reference point (wrt CoM), UnitInertia()): 6x6 matrix
default_inertia = SpatialInertia(1, [0, 0, L0/2], UnitInertia(1, 1, 1))  #This does not matter for kinematics

link_0 = plant.AddRigidBody("link_0", my_model_instance, default_inertia)

link_1 = plant.AddRigidBody("link_1", my_model_instance, default_inertia)

link_2 = plant.AddRigidBody("link_2", my_model_instance, default_inertia)

link_3 = plant.AddRigidBody("link_3", my_model_instance, default_inertia)

# plant.RegisterVisualGeometry(body, 
c =0.05
plant.RegisterVisualGeometry(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0/2]),
    Cylinder(c, L0),
    "link_0",
    RGBA_Color0)

plant.RegisterVisualGeometry(
    link_1,
    RigidTransform(RollPitchYaw(0, 0, 0), [L1/2, 0, 0]),
    Box(L1,c,c),
    "link_1",
    RGBA_Color1)

plant.RegisterVisualGeometry(
    link_2,
    RigidTransform(RollPitchYaw(0, 0, 0), [L2/2, 0, 0]),
    Box(L2, c, c),
    "link_2",
    RGBA_Color2)
plant.RegisterVisualGeometry(
    link_3,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0]),
    Box(L3, L3, L3),
    "link_3",
    RGBA_Color0)

Zero_Frame = plant.AddFrame(FixedOffsetFrame('WorldFrame',
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0])))


Joint1_Frame = plant.AddFrame(FixedOffsetFrame(
    link_0,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, L0])))

Joint2_Frame = plant.AddFrame(FixedOffsetFrame(
    link_1,
    RigidTransform(RollPitchYaw(0, np.pi/2, -np.pi/2), [L1, 0, 0])))

Joint3_Frame = plant.AddFrame(FixedOffsetFrame(
    link_2,
    RigidTransform(RollPitchYaw(-np.pi/2, 0, 0), [L2, 0, 0])))

Joint1 = plant.AddJoint(RevoluteJoint(
    name="Joint1", frame_on_parent=Joint1_Frame,
    frame_on_child=link_1.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint2 = plant.AddJoint(RevoluteJoint(
    name="Joint2", frame_on_parent=Joint2_Frame,
    frame_on_child=link_2.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))
Joint3 = plant.AddJoint(RevoluteJoint(
    name="Joint3", frame_on_parent=Joint3_Frame,
    frame_on_child=link_3.body_frame(), axis=[0, 0, 1],
    pos_lower_limit=-3.14,
    pos_upper_limit=3.14,
    damping=0.0))

plant.WeldFrames(
    frame_on_parent_P=plant.world_frame(),
    frame_on_child_C=link_0.body_frame(),
    X_PC=RigidTransform.Identity())


# add frames of interest
G = plant.AddFrame(FixedOffsetFrame('EndEffector',
    link_3,
    RigidTransform(RollPitchYaw(0, 0, 0), [0, 0, 0])))

In [107]:
# Draw RGB frames for visualization
# only one frame for each link
for body_name in ["WorldFrame", "link_1", "link_2", "EndEffector"]:
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph, 0.20, 0.008)

In [108]:
# Finalize and visualize
plant.Finalize()
renderer_name = "renderer"
scene_graph.AddRenderer(renderer_name, MakeRenderEngineVtk(RenderEngineVtkParams()))
meshcat.Delete()
meshcat_vis = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
diagram.Publish(diagram_context)

In [109]:
# write down the screw axis
import modern_robotics as mr

# step 0: compute M 
M = mr.RpToTrans(np.array([[0, 0, 1], [0, 1, 0], [-1, 0, 0]]), np.array([L1, 0, -L2]))
# step 1: compute all the screw axis

Sbar_3_0=mr.ScrewToAxis(np.array([0,0,-L2]),np.array([1,0,0]),0)
Sbar_2_0=mr.ScrewToAxis(np.array([L1,0,0]),np.array([0,-1,0]),0)
Sbar_1_0=mr.ScrewToAxis(np.array([0,0,0]),np.array([0,0,1]),0)

def myPoE(theta1,theta2,theta3):
    SbarMatrix_3_0=mr.VecTose3(Sbar_3_0)
    SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
    SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
    return mr.MatrixExp6(SbarMatrix_1_0*theta1)@mr.MatrixExp6(SbarMatrix_2_0*theta2)@mr.MatrixExp6(SbarMatrix_3_0*theta3)@M


In [207]:
# M: The homogeneous transformation matrix at home position
theta1=0
theta2=0
theta3=np.pi/6
plant.SetPositions(plant.GetMyContextFromRoot(diagram_context), my_model_instance, [theta1, theta2, theta3]) # theta1, theta2
diagram.Publish(diagram_context)

plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
T0e = G.CalcPose(plant_context, Zero_Frame)
print(T0e.GetAsMatrix4())

[[ 3.39907768e-33 -2.76012034e-33  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01  8.66025404e-01  0.00000000e+00 -3.06161700e-17]
 [-8.66025404e-01  5.00000000e-01  3.74939946e-33 -5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [208]:
Tbs = myPoE(theta1,theta2,theta3)
print(Tbs)

[[ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01  8.66025404e-01  0.00000000e+00  2.77555756e-17]
 [-8.66025404e-01  5.00000000e-01  0.00000000e+00 -5.00000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [217]:
def myGeometrixJacobian(theta1,theta2,theta3):
    SbarMatrix_2_0=mr.VecTose3(Sbar_2_0)
    SbarMatrix_1_0=mr.VecTose3(Sbar_1_0)
    # compute the SE3 for each frame
    T_01 = mr.MatrixExp6(SbarMatrix_1_0*theta1)
    T_12 = mr.MatrixExp6(SbarMatrix_2_0*theta2)
    J1 = Sbar_1_0.reshape(6,1)
    J2 = np.matmul(mr.Adjoint(T_01), Sbar_2_0.reshape(6,1))
    J3 = np.matmul(mr.Adjoint(T_01@T_12), Sbar_3_0.reshape(6,1))
    return  np.block([J1, J2, J3])


In [218]:
J = myGeometrixJacobian(theta1,theta2,theta3)
print(J)


[[ 0.   0.   1. ]
 [ 0.  -1.   0. ]
 [ 1.   0.   0. ]
 [ 0.   0.   0. ]
 [ 0.   0.  -0.5]
 [ 0.  -1.   0. ]]


In [219]:
J_G  = plant.CalcJacobianSpatialVelocity(plant_context, JacobianWrtVariable.kQDot, G, [0,0,0], Zero_Frame, Zero_Frame)
# JA_G = plant.CalcJacobianTranslationalVelocity(plant_context, JacobianWrtVariable.kQDot, G, [0,0,0], Zero_Frame, Zero_Frame)
print(J_G)
# print(JA_G)

[[ 0.00000000e+00  6.12323400e-17  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 1.00000000e+00  6.12323400e-17  3.74939946e-33]
 [ 3.06161700e-17  5.00000000e-01  0.00000000e+00]
 [ 1.00000000e+00  3.06161700e-17  0.00000000e+00]
 [ 0.00000000e+00 -1.87469973e-33  0.00000000e+00]]
[[ 3.06161700e-17  5.00000000e-01  0.00000000e+00]
 [ 1.00000000e+00  3.06161700e-17  0.00000000e+00]
 [ 0.00000000e+00 -1.87469973e-33  0.00000000e+00]]
